# Maxwell eigenvalue problem
We solve the Maxwell eigenvalue problem

$$
\int \operatorname{curl} u \, \operatorname{curl} v
= \lambda \int u v 
$$

for $u, v \; \bot \; \nabla H^1$
using a PINVIT solver from the ngsolve solvers module.

The orthogonality to gradient fields is important to eliminate the huge number of zero eigenvalues. The orthogonal sub-space is implemented using a Poisson projection:

$$
P u = u - \nabla \Delta^{-1} \operatorname{div} u
$$

The algorithm and example is take form the Phd thesis of Sabine Zaglmayr, p 145-150.

In [6]:
import netgen.gui
from netgen.csg import *
from ngsolve import *

geom = CSGeometry()

cube1 = OrthoBrick(Pnt(-1,-1,-1), Pnt(1,1,1))
cube2 = OrthoBrick(Pnt(0,0,0), Pnt(2,2,2)) 
fichera = cube1-cube2
geom.Add (fichera)

# all edges defined by intersection of faces of 
# cube2 and cube2 are marked for geometric edge refinement
geom.SingularEdge (cube2,cube2, 1)
geom.SingularPoint (cube2,cube2,cube2, 1)

mesh = Mesh(geom.GenerateMesh(maxh=0.5))
# two levels of hp-refinement
mesh.RefineHP(2, factor=0.2)
Draw (mesh)

In [7]:
SetHeapSize(100*1000*1000)

fes = HCurl(mesh, order=3)
print ("ndof =", fes.ndof)
u,v = fes.TnT()

a = BilinearForm(fes)
a += curl(u)*curl(v)*dx

m = BilinearForm(fes)
m += u*v*dx

apre = BilinearForm(fes)
apre += curl(u)*curl(v)*dx + u*v*dx
pre = Preconditioner(apre, "direct", inverse="sparsecholesky")

ndof = 27931


In [8]:
with TaskManager():
    a.Assemble()
    m.Assemble()
    apre.Assemble()

    # build gradient matrix as sparse matrix (and corresponding scalar FESpace)
    gradmat, fesh1 = fes.CreateGradient()
    
    
    gradmattrans = gradmat.CreateTranspose() # transpose sparse matrix
    math1 = gradmattrans @ m.mat @ gradmat   # multiply matrices 
    math1[0,0] += 1     # fix the 1-dim kernel
    invh1 = math1.Inverse(inverse="sparsecholesky")

    # build the Poisson projector with operator Algebra:
    proj = IdentityMatrix() - gradmat @ invh1 @ gradmattrans @ m.mat

    projpre = proj @ pre.mat

    evals, evecs = solvers.PINVIT(a.mat, m.mat, pre=projpre, num=12, maxit=20)

0 : [4.714442615762366, 18.125741568539013, 18.65984359039985, 30.09314994219186, 36.40954463032214, 42.979342303968956, 48.316457141781726, 54.58849385334882, 61.25334687172242, 70.05420062008753, 80.54664484933319, 84.62019960762365]
1 : [3.226164801496895, 6.034522799482773, 6.09820292359005, 11.745119934390791, 12.314551454030552, 13.145021966112408, 13.896095196980674, 17.0965247227754, 19.759367668803247, 21.297907661417145, 23.19215494203634, 25.89008682725409]
2 : [3.2205629431197287, 5.88664585038824, 5.890622768003521, 10.953641562697095, 11.053901602886178, 11.256681948105399, 12.738641459165656, 13.398845593761541, 13.810571349391056, 15.416989345408972, 17.628409410690082, 18.964742022299827]
3 : [3.2204891886426266, 5.880927132786673, 5.881201199696201, 10.754828955145738, 10.793726743574426, 10.91364139290134, 12.493526603482724, 12.544109421495312, 13.439872386660856, 14.132877776155906, 16.33637509577988, 16.958612546842645]
4 : [3.2204879014347934, 5.880621887780472, 

In [ ]:
print ("Eigenvalues")
for lam in evals: 
    print (lam)

In [ ]:
Draw (mesh)
gfu = GridFunction(fes, multidim=len(evecs))
for i in range(len(evecs)):
    gfu.vecs[i].data = evecs[i]
Draw (Norm(gfu), mesh, "u", sd=4)